In [7]:
import pandas as pd
import os
from datetime import datetime


In [8]:
#użytkownik podaje datę (format: RRRR-MM-DD)
data = input("Podaj datę w formacie RRRR-MM-DD (np. 2025-07-01): ").strip()

#tworzenie dynamiczych ścieżek do plików
airly_path = f"../data/processed/airly/airly_{data}.csv"
meteo_path = f"../data/processed/meteo/meteo_{data}.csv"
output_path = f"../data/combined/combined_{data}.csv"


In [13]:
def clean_airly(df: pd.DataFrame) -> pd.DataFrame:
    # Zakładamy, że kolumny to: fromDateTime, tillDateTime, name, value
    df['fromDateTime'] = pd.to_datetime(df['fromDateTime']).dt.tz_localize(None)

    # Pivot – przekształcamy dane z formatu long na wide
    df_airly_raw = pd.read_csv(airly_path, names=['fromDateTime', 'tillDateTime', 'name', 'value'])


    #zmieniamy nazwy kolumn
    df = df.rename(columns={
        'fromDateTime': 'datetime',
        'PM1': 'pm1',
        'PM25': 'pm25',
        'PM10': 'pm10',
        'PRESSURE': 'pressure',
        'HUMIDITY': 'humidity',
        'TEMPERATURE': 'temperature',
    })

    df['source'] = 'airly'
    return df

def clean_meteo(df:pd.DataFrame) -> pd.DataFrame:

    df = df.rename(columns=
        {
            'temperature_2m': 'temperature',
            'relative_humidity_2m': 'humidity',
            'windspeed_10m': 'windspeed',
        }
    )

    df['datetime'] = pd.to_datetime(df['time'])

    df['source'] = 'meteo'

    cols = ['datetime', 'temperature', 'humidity', 'windspeed', 'source']
    df = df[cols]

    return df




In [14]:
try: 
    df_airly_raw = pd.read_csv(airly_path)
    df_meteo_raw = pd.read_csv(meteo_path)

    df_airly = clean_airly(df_airly_raw)
    df_meteo = clean_meteo(df_meteo_raw)

    df_combined = pd.concat([df_airly, df_meteo], ignore_index=True)
    df_combined = df_combined.sort_values("datetime")

    #tworzenie katalogu, jeśli nie istnieje
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    #zapisanie DataFrame do pliku CSV
    df_combined.to_csv(output_path, index=False)

    print(f"✅ Dane połączone i zapisane do: {output_path} ✅")

except FileNotFoundError as e:
    print(f"❌ Błąd: Plik {e.filename} nie został znaleziony. Sprawdź ścieżki do plików: {e}")
except pd.errors.EmptyDataError:
    print("❌ Błąd: Plik jest pusty lub nie zawiera danych.")
except Exception as e:
    print(f"❌ Wystąpił błąd: {e}. Sprawdź, czy pliki są poprawne i czy nie ma problemów z formatem danych.")

✅ Dane połączone i zapisane do: ../data/combined/combined_2025-07-01.csv ✅
